In [ ]:

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit


RANDOM FOREST

In [4]:
class SklearnEarthquakePredictor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=23, max_depth=12, min_samples_split=7, 
                 shiftnum=3, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.shiftnum = shiftnum
        self.max_features = max_features
        self.random_state = random_state
        self.target_names = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']#tahmin yapacağımız kısımlar
        self.rf_model = RandomForestRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            max_features=self.max_features,
            random_state=self.random_state,
            n_jobs=-1
        )
    
    def prepare(self, df):
        dfshifts = df.copy()
        for i in range(1, self.shiftnum + 1):
            dfshifts[f'mag{i}'] = df['mag'].shift(i)
            dfshifts[f'depth{i}'] = df['depth'].shift(i)
            dfshifts[f'lat{i}'] = df['latitude'].shift(i)
            dfshifts[f'lon{i}'] = df['longitude'].shift(i)
        feature_cols = [f'{v}{i}' for i in range(1, self.shiftnum + 1) 
                       for v in ('mag', 'depth', 'lat', 'lon')]
        target_cols = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
        available_feature_cols = [col for col in feature_cols if col in dfshifts.columns]#bu kontrolü eklemek yapay zeka yardımıyla yaptığım bir şeydi
        available_target_cols = [col for col in target_cols if col in dfshifts.columns]#öncesinde hepsinde sadece feature_cols ve target_cols yazıyordu. 
        X = dfshifts[available_feature_cols]
        Y = dfshifts[available_target_cols]
        combined = pd.concat([X, Y], axis=1)
        combined = combined.dropna()
        Xnew = combined[available_feature_cols].values
        Ynew = combined[available_target_cols].values
        return Xnew, Ynew
    
    def fit(self, X, y=None):
        df = X
        X_processed, Y_processed = self.prepare(df)
        self.rf_model.fit(X_processed, Y_processed)
        return self
    
    def predict(self, df):
        X_processed, _ = self.prepare(df)
        predictions = self.rf_model.predict(X_processed)  
        return predictions
    

XGBOOST

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

class XGBoostEarthquakePredictor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=100, max_depth=6, learning_rate=0.1, 
                 subsample=0.8, colsample_bytree=0.8, min_child_weight=1,
                 reg_alpha=0, reg_lambda=1, shiftnum=3, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.min_child_weight = min_child_weight
        self.reg_alpha = reg_alpha
        self.reg_lambda = reg_lambda
        self.shiftnum = shiftnum
        self.random_state = random_state
        self.target_names = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
        
        
        self.fitted_feature_columns = None
        
        self.xgb_model = XGBRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            learning_rate=self.learning_rate,
            subsample=self.subsample,
            colsample_bytree=self.colsample_bytree,
            min_child_weight=self.min_child_weight,
            reg_alpha=self.reg_alpha,
            reg_lambda=self.reg_lambda,
            random_state=self.random_state,
            n_jobs=-1,
            verbosity=0
        )
    
    def _preprocess_data(self, df, is_training=False):
        df_processed = df.copy()
        columns_to_drop = ['id', 'place', 'updated', 'locationSource']
        columns_to_drop = [col for col in columns_to_drop if col in df_processed.columns]
        if columns_to_drop:
            df_processed = df_processed.drop(columns=columns_to_drop)
        categorical_cols = ['magType', 'type', 'status', 'net']
        available_categorical_cols = [col for col in categorical_cols if col in df_processed.columns]
        
        if available_categorical_cols:
            # One-hot encoding için https://www.datacamp.com/tutorial/category/ai den yazı okudum,kodlamada yer yer yapay zekadan yardım aldım
            df_processed = pd.get_dummies(df_processed, columns=available_categorical_cols, 
                                        prefix=available_categorical_cols, dummy_na=False)
        
        return df_processed
    
    def prepare(self, df, is_training=False):
        df_processed = self._preprocess_data(df, is_training)
        
        dfshifts = df_processed.copy()
        
        for i in range(1, self.shiftnum + 1):
            dfshifts[f'mag{i}'] = df_processed['mag'].shift(i)
            dfshifts[f'depth{i}'] = df_processed['depth'].shift(i)
            dfshifts[f'lat{i}'] = df_processed['latitude'].shift(i)
            dfshifts[f'lon{i}'] = df_processed['longitude'].shift(i)
        
        
        base_feature_cols = [f'{v}{i}' for i in range(1, self.shiftnum + 1) 
                           for v in ('mag', 'depth', 'lat', 'lon')]
        
        
        target_cols = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
        #Prepare metodunun  in buradan sonrasında yapay zeka X ve Y yi oluşturduğumuz yere kadarki 10 satırda yardım etti 
    
        available_base_features = [col for col in base_feature_cols if col in dfshifts.columns]
        available_target_cols = [col for col in target_cols if col in dfshifts.columns]
        
       
        categorical_feature_cols = [col for col in dfshifts.columns 
                                   if any(cat in col for cat in ['magType_', 'type_', 'status_', 'net_'])]
        
       
        if is_training:
            self.fitted_feature_columns = available_base_features + categorical_feature_cols
            all_feature_cols = self.fitted_feature_columns
        else:
            for col in self.fitted_feature_columns:
                if col not in dfshifts.columns:
                    dfshifts[col] = 0
                    
            all_feature_cols = self.fitted_feature_columns
        
        X = dfshifts[all_feature_cols]
        Y = dfshifts[available_target_cols]
        
       
        combined = pd.concat([X, Y], axis=1)
        combined = combined.dropna()
        

        Xnew = combined[all_feature_cols].values
        Ynew = combined[available_target_cols].values
        
        
        
        return Xnew, Ynew
    
    def fit(self, X, y=None):
        df = X
        X_processed, Y_processed = self.prepare(df, is_training=True)
        self.xgb_model.fit(X_processed, Y_processed)
        return self
    
    def predict(self, df):
        X_processed, _ = self.prepare(df, is_training=False)
        predictions = self.xgb_model.predict(X_processed)
        return predictions

MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

class MLPEarthquakePredictor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_layer_sizes=(100,), activation='relu', solver='adam',
                 alpha=0.0001, learning_rate_init=0.001, max_iter=1000,
                 shiftnum=3, random_state=None):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.solver = solver
        self.alpha = alpha
        self.learning_rate_init = learning_rate_init
        self.max_iter = max_iter
        self.shiftnum = shiftnum
        self.random_state = random_state
        self.target_names = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
        
        # Scaler ve imputer'ları saklamak için
        self.feature_scaler = None
        self.target_scaler = None
        self.feature_imputer = None
        self.target_imputer = None
        self.fitted_feature_columns = None
        
        self.mlp_model = MLPRegressor(
            hidden_layer_sizes=self.hidden_layer_sizes,
            activation=self.activation,
            solver=self.solver,
            alpha=self.alpha,
            learning_rate_init=self.learning_rate_init,
            max_iter=self.max_iter,
            random_state=self.random_state
        )
    
    def _preprocess_data(self, df, is_training=False):
        df_processed = df.copy()
        
        # Gereksiz sütunları kaldır
        columns_to_drop = ['id', 'place', 'updated', 'locationSource']
        columns_to_drop = [col for col in columns_to_drop if col in df_processed.columns]
        if columns_to_drop:
            df_processed = df_processed.drop(columns=columns_to_drop)
        
        # Kategorik sütunlar için one-hot encoding
        categorical_cols = ['magType', 'type', 'status', 'net']
        available_categorical_cols = [col for col in categorical_cols if col in df_processed.columns]
        
        if available_categorical_cols:
            df_processed = pd.get_dummies(df_processed, columns=available_categorical_cols, 
                                        prefix=available_categorical_cols, dummy_na=False)
        
        return df_processed
    
    def prepare(self, df, is_training=False):
        df_processed = self._preprocess_data(df, is_training)
        dfshifts = df_processed.copy()
        for i in range(1, self.shiftnum + 1):
            dfshifts[f'mag{i}'] = df_processed['mag'].shift(i)
            dfshifts[f'depth{i}'] = df_processed['depth'].shift(i)
            dfshifts[f'lat{i}'] = df_processed['latitude'].shift(i)
            dfshifts[f'lon{i}'] = df_processed['longitude'].shift(i)
        
        base_feature_cols = [f'{v}{i}' for i in range(1, self.shiftnum + 1) 
                           for v in ('mag', 'depth', 'lat', 'lon')]
        
        target_cols = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
        
        available_base_features = [col for col in base_feature_cols if col in dfshifts.columns]
        available_target_cols = [col for col in target_cols if col in dfshifts.columns]
        
        categorical_feature_cols = [col for col in dfshifts.columns 
                                   if any(cat in col for cat in ['magType_', 'type_', 'status_', 'net_'])]
        
        if is_training:
            self.fitted_feature_columns = available_base_features + categorical_feature_cols
            all_feature_cols = self.fitted_feature_columns
        else:
            for col in self.fitted_feature_columns:
                if col not in dfshifts.columns:
                    dfshifts[col] = 0
                    
            all_feature_cols = self.fitted_feature_columns
        
        X = dfshifts[all_feature_cols]
        Y = dfshifts[available_target_cols]
        
        
        combined = pd.concat([X, Y], axis=1)
        combined = combined.dropna()
        
        Xnew = combined[all_feature_cols]
        Ynew = combined[available_target_cols]
        
        
        if is_training:
            #
            self.feature_imputer = SimpleImputer(strategy='mean')
            self.feature_scaler = MinMaxScaler()
            
            # Target imputer ve scaler
            self.target_imputer = SimpleImputer(strategy='mean')
            self.target_scaler = MinMaxScaler()
            
            # Fit ve transform
            Xnew_imputed = self.feature_imputer.fit_transform(Xnew)
            Xnew_scaled = self.feature_scaler.fit_transform(Xnew_imputed)
            
            Ynew_imputed = self.target_imputer.fit_transform(Ynew)
            Ynew_scaled = self.target_scaler.fit_transform(Ynew_imputed)
            
        else:
            # Transform only
            Xnew_imputed = self.feature_imputer.transform(Xnew)
            Xnew_scaled = self.feature_scaler.transform(Xnew_imputed)
            
            Ynew_imputed = self.target_imputer.transform(Ynew)
            Ynew_scaled = self.target_scaler.transform(Ynew_imputed)
        
        return Xnew_scaled, Ynew_scaled
    
    def fit(self, X, y=None):
        df = X
        X_processed, Y_processed = self.prepare(df, is_training=True)
        self.mlp_model.fit(X_processed, Y_processed)
        return self
    
    def predict(self, df):
        X_processed, _ = self.prepare(df, is_training=False)
        predictions_scaled = self.mlp_model.predict(X_processed)
        predictions = self.target_scaler.inverse_transform(predictions_scaled)#0-1 aralığından normale dönüş
        return predictions
